In [118]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
from tqdm.notebook import tqdm_notebook

import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier, plot_tree 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_curve, auc , f1_score, recall_score, accuracy_score
from sklearn.metrics import confusion_matrix, classification_report 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer , StandardScaler

## Read data 

In [119]:
# data train include provider ID and labels
Train = pd.read_csv("data/Train-1542865627584.csv")

# include beneficiary KYC details like health conditions,region
Train_Beneficiarydata = pd.read_csv("data/Train_Beneficiarydata-1542865627584.csv")

# include the claims filed for those patients who are admitted in the hospitals
Train_Inpatientdata=pd.read_csv("data/Train_Inpatientdata-1542865627584.csv")

# include the claims filed for those patients who visit hospitals and not admitted in it.
Train_Outpatientdata=pd.read_csv("data/Train_Outpatientdata-1542865627584.csv")


In [120]:
print("Train\n", Train.shape)
print("Train_Beneficiarydata\n", Train_Beneficiarydata.shape)
print("Train_Inpatientdata\n", Train_Inpatientdata.shape)
print("Train_Outpatientdata\n", Train_Outpatientdata.shape)


Train
 (5410, 2)
Train_Beneficiarydata
 (138556, 25)
Train_Inpatientdata
 (40474, 30)
Train_Outpatientdata
 (517737, 27)


## Functions

In [121]:
def agg_calculation(dataset,
                        groupby_col = '',
                        agg_col = '',
                        sort_col = '',
                        agg_method_lst = [],
                        col_name_lst = []
                       ):
    """This function makes aggregation on specific column"""

    grouped = dataset.groupby(groupby_col)[agg_col].agg(agg_method_lst).reset_index()
    grouped.columns = col_name_lst
    grouped = grouped.sort_values(by = [sort_col], ascending=False).reset_index(drop = True)

    return grouped


In [122]:
def get_top_5_codes(group, col=''):
    top_5 = group[col].value_counts().nlargest(5).index.tolist()
    return top_5

In [123]:
# Function to count occurrences of values across multiple columns and find top 5
def top_5_across_columns(df, columns):
    counts = {}
    for col in columns:
        for val in df[col]:
            if pd.notnull(val):
                if val in counts:
                    counts[val] += 1
                else:
                    counts[val] = 1
    top_5 = sorted(counts, key=counts.get, reverse=True)[:5]
    return top_5

In [124]:
#check for same physician in all three physician columns
def physician_same(row):
    atten_oper = row['AttendingPhysician'] == row['OperatingPhysician']
    oper_other = row['OperatingPhysician'] == row['OtherPhysician']
    atten_other = row['AttendingPhysician'] == row['OtherPhysician']
    
    # atten = oper = other
    if atten_oper == True and oper_other == True:
        return 0
    
    # atten = oper != other
    elif atten_oper == True and oper_other == False:
        return 1
    
    # atten = other != oper
    elif atten_other == True and oper_other == False:
        return 2
    
    # atten != other = oper
    elif atten_other == False and oper_other == True:
        return 3
    
    # atten != other != oper
    else:
        return 4

## Create Features

### Inpatient

In [125]:
print('inpatient columns:\n',  Train_Inpatientdata.columns)
Train_Inpatientdata.head(5)

inpatient columns:
 Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
       'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5',
       'ClmProcedureCode_6'],
      dtype='object')


,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,5119,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN


In [126]:
# check duplicates
print(len(list(Train_Inpatientdata.ClaimID.value_counts()==1)))
print(len(list(Train_Inpatientdata['ClaimID'].unique())))

40474
40474


#### 1. Num of inpatient claims for each provider

In [127]:
df1 = agg_calculation(Train_Inpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'ClaimID',
                    sort_col = 'ip_claims_num',
                    agg_method_lst = ['count'],
                    col_name_lst = ['Provider', 'ip_claims_num']
                       )
df1

,Provider,ip_claims_num
0,PRV52019,516
1,PRV55462,386
2,PRV54367,322
3,PRV53706,282
4,PRV55209,275
...,...,...
2087,PRV53515,1
2088,PRV53516,1
2089,PRV53600,1
2090,PRV56474,1


#### 2. Num of inpatient patients for each provider

In [128]:
df2 = agg_calculation(Train_Inpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'BeneID',
                    sort_col = 'ip_patients_num',
                    agg_method_lst = ['nunique'],
                    col_name_lst = ['Provider', 'ip_patients_num']
                       )
df_merged = df1.merge(df2, how = 'outer', on = 'Provider')
df_merged

,Provider,ip_claims_num,ip_patients_num
0,PRV52019,516,458
1,PRV55462,386,308
2,PRV54367,322,279
3,PRV53706,282,262
4,PRV55209,275,243
...,...,...,...
2087,PRV53515,1,1
2088,PRV53516,1,1
2089,PRV53600,1,1
2090,PRV56474,1,1


#### 3. Total inpatient reimbursement for each provider

In [129]:
df3 = agg_calculation(Train_Inpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'InscClaimAmtReimbursed',
                    sort_col = 'ip_reimbursed',
                    agg_method_lst = ['sum'],
                    col_name_lst = ['Provider', 'ip_reimbursed']
                       )
df_merged = df_merged.merge(df3, how = 'outer', on = 'Provider')
df_merged

,Provider,ip_claims_num,ip_patients_num,ip_reimbursed
0,PRV52019,516,458,5580870
1,PRV55462,386,308,4260100
2,PRV54367,322,279,3040900
3,PRV53706,282,262,2776000
4,PRV55209,275,243,2756100
...,...,...,...,...
2087,PRV53515,1,1,14000
2088,PRV53516,1,1,3000
2089,PRV53600,1,1,4000
2090,PRV56474,1,1,7000


#### 4. Total inpatient deductible amount for each provider

In [130]:
df4 = agg_calculation(Train_Inpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'DeductibleAmtPaid',
                    sort_col = 'ip_deductible',
                    agg_method_lst = ['sum'],
                    col_name_lst = ['Provider', 'ip_deductible']
                       )
df_merged = df_merged.merge(df4, how = 'outer', on = 'Provider')
df_merged

,Provider,ip_claims_num,ip_patients_num,ip_reimbursed,ip_deductible
0,PRV52019,516,458,5580870,536136.0
1,PRV55462,386,308,4260100,405840.0
2,PRV54367,322,279,3040900,334284.0
3,PRV53706,282,262,2776000,296904.0
4,PRV55209,275,243,2756100,289428.0
...,...,...,...,...,...
2087,PRV53515,1,1,14000,1068.0
2088,PRV53516,1,1,3000,1068.0
2089,PRV53600,1,1,4000,1068.0
2090,PRV56474,1,1,7000,1068.0


#### 5. Total claim length for each provider

In [131]:
startdate = pd.to_datetime(Train_Inpatientdata.ClaimStartDt)
enddate = pd.to_datetime(Train_Inpatientdata.ClaimEndDt)
Train_Inpatientdata['Cperiod'] = (enddate - startdate).dt.days+1

df5 = agg_calculation(Train_Inpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'Cperiod',
                    sort_col = 'ip_cperiod',
                    agg_method_lst = ['sum'],
                    col_name_lst = ['Provider', 'ip_cperiod']
                       )
df_merged = df_merged.merge(df5, how = 'outer', on = 'Provider')
df_merged

,Provider,ip_claims_num,ip_patients_num,ip_reimbursed,ip_deductible,ip_cperiod
0,PRV52019,516,458,5580870,536136.0,3560
1,PRV55462,386,308,4260100,405840.0,2682
2,PRV54367,322,279,3040900,334284.0,2052
3,PRV53706,282,262,2776000,296904.0,1892
4,PRV55209,275,243,2756100,289428.0,1744
...,...,...,...,...,...,...
2087,PRV53515,1,1,14000,1068.0,6
2088,PRV53516,1,1,3000,1068.0,9
2089,PRV53600,1,1,4000,1068.0,3
2090,PRV56474,1,1,7000,1068.0,13


#### 6. Total hospital stay for each provider

In [132]:
startdate = pd.to_datetime(Train_Inpatientdata.AdmissionDt)
enddate = pd.to_datetime(Train_Inpatientdata.DischargeDt)
Train_Inpatientdata['HospitalStay'] = (enddate - startdate).dt.days+1

df6 = agg_calculation(Train_Inpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'HospitalStay',
                    sort_col = 'ip_hperiod',
                    agg_method_lst = ['sum'],
                    col_name_lst = ['Provider', 'ip_hperiod']
                       )
df_merged = df_merged.merge(df6, how = 'outer', on = 'Provider')
df_merged

,Provider,ip_claims_num,ip_patients_num,ip_reimbursed,ip_deductible,ip_cperiod,ip_hperiod
0,PRV52019,516,458,5580870,536136.0,3560,3568
1,PRV55462,386,308,4260100,405840.0,2682,2682
2,PRV54367,322,279,3040900,334284.0,2052,2052
3,PRV53706,282,262,2776000,296904.0,1892,1919
4,PRV55209,275,243,2756100,289428.0,1744,1744
...,...,...,...,...,...,...,...
2087,PRV53515,1,1,14000,1068.0,6,6
2088,PRV53516,1,1,3000,1068.0,9,9
2089,PRV53600,1,1,4000,1068.0,3,3
2090,PRV56474,1,1,7000,1068.0,13,13


#### 7. Calculate a relationship score among three physcian columns for each claim and average on each provider

In [133]:
Train_Inpatientdata['phy_same'] = Train_Inpatientdata.apply(physician_same, axis=1)

In [134]:
df7 = agg_calculation(Train_Inpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'phy_same',
                    sort_col = 'ip_phy_same',
                    agg_method_lst = ['mean'],
                    col_name_lst = ['Provider', 'ip_phy_same']
                       )
df_merged = df_merged.merge(df7, how = 'outer', on = 'Provider')
df_merged

,Provider,ip_claims_num,ip_patients_num,ip_reimbursed,ip_deductible,ip_cperiod,ip_hperiod,ip_phy_same
0,PRV52019,516,458,5580870,536136.0,3560,3568,3.614341
1,PRV55462,386,308,4260100,405840.0,2682,2682,4.000000
2,PRV54367,322,279,3040900,334284.0,2052,2052,2.611801
3,PRV53706,282,262,2776000,296904.0,1892,1919,3.659574
4,PRV55209,275,243,2756100,289428.0,1744,1744,2.330909
...,...,...,...,...,...,...,...,...
2087,PRV53515,1,1,14000,1068.0,6,6,4.000000
2088,PRV53516,1,1,3000,1068.0,9,9,4.000000
2089,PRV53600,1,1,4000,1068.0,3,3,1.000000
2090,PRV56474,1,1,7000,1068.0,13,13,4.000000


#### 8. Number of different physicians who appear on a claim and average on each provider

In [135]:
#encoding types of physicians into numeric values
Train_Inpatientdata[['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']] = np.where(Train_Inpatientdata[['AttendingPhysician','OperatingPhysician','OtherPhysician']].isnull(), 0, 1)

# number of different physicians who attend a patient
Train_Inpatientdata['N_Types_Physicians'] = Train_Inpatientdata['AttendingPhysician'] +  Train_Inpatientdata['OperatingPhysician'] + Train_Inpatientdata['OtherPhysician']


In [136]:
df8 = agg_calculation(Train_Inpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'N_Types_Physicians',
                    sort_col = 'ip_N_Types_Physicians',
                    agg_method_lst = ['mean'],
                    col_name_lst = ['Provider', 'ip_N_Types_Physicians']
                       )
df_merged = df_merged.merge(df8, how = 'outer', on = 'Provider')
df_merged

,Provider,ip_claims_num,ip_patients_num,ip_reimbursed,ip_deductible,ip_cperiod,ip_hperiod,ip_phy_same,ip_N_Types_Physicians
0,PRV52019,516,458,5580870,536136.0,3560,3568,3.614341,1.812016
1,PRV55462,386,308,4260100,405840.0,2682,2682,4.000000,1.582902
2,PRV54367,322,279,3040900,334284.0,2052,2052,2.611801,1.577640
3,PRV53706,282,262,2776000,296904.0,1892,1919,3.659574,1.723404
4,PRV55209,275,243,2756100,289428.0,1744,1744,2.330909,1.552727
...,...,...,...,...,...,...,...,...,...
2087,PRV53515,1,1,14000,1068.0,6,6,4.000000,2.000000
2088,PRV53516,1,1,3000,1068.0,9,9,4.000000,1.000000
2089,PRV53600,1,1,4000,1068.0,3,3,1.000000,2.000000
2090,PRV56474,1,1,7000,1068.0,13,13,4.000000,1.000000


#### 9. top 5 ClmAdmitDiagnosisCode for each provider

In [137]:
df9 = Train_Inpatientdata.groupby('Provider').apply(lambda x: get_top_5_codes(x, 'ClmAdmitDiagnosisCode')).reset_index(name='Top_5_ClmAdmitDiagnosisCode_ip')
df9

,Provider,Top_5_ClmAdmitDiagnosisCode_ip
0,PRV51001,"[29623, 0389, 5362, 41401, 80121]"
1,PRV51003,"[78605, 4280, 42789, 78650, 78701]"
2,PRV51007,"[78097, 29570, 29633]"
3,PRV51008,"[7837, 920]"
4,PRV51011,[78906]
...,...,...
2087,PRV57719,"[68110, 5789, 78703, 4271]"
2088,PRV57728,[42789]
2089,PRV57729,"[486, 43491, 78605, 5990, 41401]"
2090,PRV57732,"[5119, 7907, 71535, V5789, 7242]"


In [138]:
df_merged = df_merged.merge(df9, how = 'outer', on = 'Provider')
df_merged

,Provider,ip_claims_num,ip_patients_num,ip_reimbursed,ip_deductible,ip_cperiod,ip_hperiod,ip_phy_same,ip_N_Types_Physicians,Top_5_ClmAdmitDiagnosisCode_ip
0,PRV52019,516,458,5580870,536136.0,3560,3568,3.614341,1.812016,"[78650, 486, 42731, 78605, 4280]"
1,PRV55462,386,308,4260100,405840.0,2682,2682,4.000000,1.582902,"[78650, 486, 78605, V5789, 78097]"
2,PRV54367,322,279,3040900,334284.0,2052,2052,2.611801,1.577640,"[78650, 78605, V5789, 42731, 78659]"
3,PRV53706,282,262,2776000,296904.0,1892,1919,3.659574,1.723404,"[78605, 486, 4280, 78650, 71536]"
4,PRV55209,275,243,2756100,289428.0,1744,1744,2.330909,1.552727,"[78650, 486, 4280, 78605, 0389]"
...,...,...,...,...,...,...,...,...,...,...
2087,PRV53515,1,1,14000,1068.0,6,6,4.000000,2.000000,[72402]
2088,PRV53516,1,1,3000,1068.0,9,9,4.000000,1.000000,[49322]
2089,PRV53600,1,1,4000,1068.0,3,3,1.000000,2.000000,[V5413]
2090,PRV56474,1,1,7000,1068.0,13,13,4.000000,1.000000,[486]


#### 10. top 5 DiagnosisGroupCode for each provider

In [139]:
df10 = Train_Inpatientdata.groupby('Provider').apply(lambda x: get_top_5_codes(x, 'DiagnosisGroupCode')).reset_index(name='Top_5_DiagnosisGroupCode_ip')
df10

,Provider,Top_5_DiagnosisGroupCode_ip
0,PRV51001,"[882, 864, 353, 245, 062]"
1,PRV51003,"[203, 262, 241, 222, 627]"
2,PRV51007,"[085, 886, 887]"
3,PRV51008,"[623, 095]"
4,PRV51011,[414]
...,...,...
2087,PRV57719,"[605, 684, 341, 005]"
2088,PRV57728,[257]
2089,PRV57729,"[288, 183, 103, 669, 082]"
2090,PRV57732,"[941, 189, 473, 414, 939]"


In [140]:
df_merged = df_merged.merge(df10, how = 'outer', on = 'Provider')
df_merged

,Provider,ip_claims_num,ip_patients_num,ip_reimbursed,ip_deductible,ip_cperiod,ip_hperiod,ip_phy_same,ip_N_Types_Physicians,Top_5_ClmAdmitDiagnosisCode_ip,Top_5_DiagnosisGroupCode_ip
0,PRV52019,516,458,5580870,536136.0,3560,3568,3.614341,1.812016,"[78650, 486, 42731, 78605, 4280]","[181, 627, 205, 882, 694]"
1,PRV55462,386,308,4260100,405840.0,2682,2682,4.000000,1.582902,"[78650, 486, 78605, V5789, 78097]","[167, 950, 866, 304, 217]"
2,PRV54367,322,279,3040900,334284.0,2052,2052,2.611801,1.577640,"[78650, 78605, V5789, 42731, 78659]","[204, 196, 232, 941, 255]"
3,PRV53706,282,262,2776000,296904.0,1892,1919,3.659574,1.723404,"[78605, 486, 4280, 78650, 71536]","[864, 241, 245, 264, 949]"
4,PRV55209,275,243,2756100,289428.0,1744,1744,2.330909,1.552727,"[78650, 486, 4280, 78605, 0389]","[883, 183, 871, 887, 866]"
...,...,...,...,...,...,...,...,...,...,...,...
2087,PRV53515,1,1,14000,1068.0,6,6,4.000000,2.000000,[72402],[476]
2088,PRV53516,1,1,3000,1068.0,9,9,4.000000,1.000000,[49322],[207]
2089,PRV53600,1,1,4000,1068.0,3,3,1.000000,2.000000,[V5413],[514]
2090,PRV56474,1,1,7000,1068.0,13,13,4.000000,1.000000,[486],[188]


#### 11. top 5 ClmDiagnosisCode for each provider

In [141]:
diagnosis_col = ['ClmDiagnosisCode_1','ClmDiagnosisCode_2','ClmDiagnosisCode_3','ClmDiagnosisCode_4','ClmDiagnosisCode_5','ClmDiagnosisCode_6','ClmDiagnosisCode_7','ClmDiagnosisCode_8','ClmDiagnosisCode_9','ClmDiagnosisCode_10']

df11 = Train_Inpatientdata.groupby('Provider').apply(lambda x: top_5_across_columns(x, diagnosis_col)).reset_index(name='Top_5_ClmDiagnosisCode_ip')
df11

,Provider,Top_5_ClmDiagnosisCode_ip
0,PRV51001,"[2724, 53081, 29650, 0388, 56211]"
1,PRV51003,"[4019, 4280, 2724, 41401, 5990]"
2,PRV51007,"[3310, 29620, 29622, 25000, 4019]"
3,PRV51008,"[27651, 1911, 4439, 2948, V1046]"
4,PRV51011,"[5770, 32723, 311, 5771, 2449]"
...,...,...
2087,PRV57719,"[25000, 6827, 5849, 56211, 0389]"
2088,PRV57728,"[42732, 49390, 2729, 78791, 7906]"
2089,PRV57729,"[4280, 2724, 41401, 4019, 5990]"
2090,PRV57732,"[4019, 486, 53081, 2724, 40390]"


In [142]:
df_merged = df_merged.merge(df11, how = 'outer', on = 'Provider')
df_merged

,Provider,ip_claims_num,ip_patients_num,ip_reimbursed,ip_deductible,ip_cperiod,ip_hperiod,ip_phy_same,ip_N_Types_Physicians,Top_5_ClmAdmitDiagnosisCode_ip,Top_5_DiagnosisGroupCode_ip,Top_5_ClmDiagnosisCode_ip
0,PRV52019,516,458,5580870,536136.0,3560,3568,3.614341,1.812016,"[78650, 486, 42731, 78605, 4280]","[181, 627, 205, 882, 694]","[4019, 25000, 2724, 42731, 4280]"
1,PRV55462,386,308,4260100,405840.0,2682,2682,4.000000,1.582902,"[78650, 486, 78605, V5789, 78097]","[167, 950, 866, 304, 217]","[4019, 2724, 25000, 41401, 42731]"
2,PRV54367,322,279,3040900,334284.0,2052,2052,2.611801,1.577640,"[78650, 78605, V5789, 42731, 78659]","[204, 196, 232, 941, 255]","[4019, 2724, 25000, 41401, 53081]"
3,PRV53706,282,262,2776000,296904.0,1892,1919,3.659574,1.723404,"[78605, 486, 4280, 78650, 71536]","[864, 241, 245, 264, 949]","[4019, 2724, 41401, 42731, 25000]"
4,PRV55209,275,243,2756100,289428.0,1744,1744,2.330909,1.552727,"[78650, 486, 4280, 78605, 0389]","[883, 183, 871, 887, 866]","[4019, 2724, 4280, 25000, 42731]"
...,...,...,...,...,...,...,...,...,...,...,...,...
2087,PRV53515,1,1,14000,1068.0,6,6,4.000000,2.000000,[72402],[476],"[72402, 51883, 71690, 49320, 3051]"
2088,PRV53516,1,1,3000,1068.0,9,9,4.000000,1.000000,[49322],[207],"[49322, V433, 2449, 78650, 30522]"
2089,PRV53600,1,1,4000,1068.0,3,3,1.000000,2.000000,[V5413],[514],"[99666, 27801, V4365, 0414, 2724]"
2090,PRV56474,1,1,7000,1068.0,13,13,4.000000,1.000000,[486],[188],"[486, 04112, 71690, 5307, 2440]"


#### 12. top 5 ClmProcedureCode for each provider

In [143]:
procedure_col = ['ClmProcedureCode_1','ClmProcedureCode_2','ClmProcedureCode_3','ClmProcedureCode_4','ClmProcedureCode_5','ClmProcedureCode_6']

df12 = Train_Inpatientdata.groupby('Provider').apply(lambda x: top_5_across_columns(x, procedure_col)).reset_index(name='Top_5_ClmProcedureCode_ip')
df12

,Provider,Top_5_ClmProcedureCode_ip
0,PRV51001,"[3521.0, 8659.0, 2724.0]"
1,PRV51003,"[4019.0, 8622.0, 3491.0, 3995.0, 3893.0]"
2,PRV51007,[8627.0]
3,PRV51008,"[8954.0, 159.0]"
4,PRV51011,[]
...,...,...
2087,PRV57719,"[4516.0, 131.0, 8604.0]"
2088,PRV57728,[]
2089,PRV57729,"[3722.0, 3613.0, 9904.0, 8841.0, 66.0]"
2090,PRV57732,"[8154.0, 9960.0, 4233.0, 9338.0, 8838.0]"


In [144]:
df_merged = df_merged.merge(df12, how = 'outer', on = 'Provider')
df_merged

,Provider,ip_claims_num,ip_patients_num,ip_reimbursed,ip_deductible,ip_cperiod,ip_hperiod,ip_phy_same,ip_N_Types_Physicians,Top_5_ClmAdmitDiagnosisCode_ip,Top_5_DiagnosisGroupCode_ip,Top_5_ClmDiagnosisCode_ip,Top_5_ClmProcedureCode_ip
0,PRV52019,516,458,5580870,536136.0,3560,3568,3.614341,1.812016,"[78650, 486, 42731, 78605, 4280]","[181, 627, 205, 882, 694]","[4019, 25000, 2724, 42731, 4280]","[4019.0, 8154.0, 2724.0, 3893.0, 9904.0]"
1,PRV55462,386,308,4260100,405840.0,2682,2682,4.000000,1.582902,"[78650, 486, 78605, V5789, 78097]","[167, 950, 866, 304, 217]","[4019, 2724, 25000, 41401, 42731]","[4019.0, 9904.0, 66.0, 8154.0, 3893.0]"
2,PRV54367,322,279,3040900,334284.0,2052,2052,2.611801,1.577640,"[78650, 78605, V5789, 42731, 78659]","[204, 196, 232, 941, 255]","[4019, 2724, 25000, 41401, 53081]","[4019.0, 9904.0, 2724.0, 66.0, 8154.0]"
3,PRV53706,282,262,2776000,296904.0,1892,1919,3.659574,1.723404,"[78605, 486, 4280, 78650, 71536]","[864, 241, 245, 264, 949]","[4019, 2724, 41401, 42731, 25000]","[4019.0, 8154.0, 66.0, 2724.0, 9904.0]"
4,PRV55209,275,243,2756100,289428.0,1744,1744,2.330909,1.552727,"[78650, 486, 4280, 78605, 0389]","[883, 183, 871, 887, 866]","[4019, 2724, 4280, 25000, 42731]","[4019.0, 4513.0, 9904.0, 8154.0, 3893.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,PRV53515,1,1,14000,1068.0,6,6,4.000000,2.000000,[72402],[476],"[72402, 51883, 71690, 49320, 3051]","[309.0, 1749.0]"
2088,PRV53516,1,1,3000,1068.0,9,9,4.000000,1.000000,[49322],[207],"[49322, V433, 2449, 78650, 30522]",[]
2089,PRV53600,1,1,4000,1068.0,3,3,1.000000,2.000000,[V5413],[514],"[99666, 27801, V4365, 0414, 2724]",[8944.0]
2090,PRV56474,1,1,7000,1068.0,13,13,4.000000,1.000000,[486],[188],"[486, 04112, 71690, 5307, 2440]",[]


### Outpatient

In [145]:
print('outpatient columns:\n',  Train_Outpatientdata.columns)
Train_Outpatientdata.head(5)

outpatient columns:
 Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode'],
      dtype='object')


,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,56409
1,BENE11003,CLM189947,2009-02-12,2009-02-12,PRV57610,80,PHY362868,NaN,NaN,6115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,79380
2,BENE11003,CLM438021,2009-06-27,2009-06-27,PRV57595,10,PHY328821,NaN,NaN,2723,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,71947


In [146]:
#show null values percentage
print('null value percentage:\n', Train_Outpatientdata.isnull().mean()*100)

#figure out high percentage columns
nullPercent = Train_Outpatientdata.isnull().mean()*100
filtered_columns = nullPercent[nullPercent > 70].index
print(filtered_columns)

null value percentage:
 BeneID                      0.000000
ClaimID                     0.000000
ClaimStartDt                0.000000
ClaimEndDt                  0.000000
Provider                    0.000000
InscClaimAmtReimbursed      0.000000
AttendingPhysician          0.269635
OperatingPhysician         82.497484
OtherPhysician             62.327205
ClmDiagnosisCode_1          2.018979
ClmDiagnosisCode_2         37.737307
ClmDiagnosisCode_3         60.741264
ClmDiagnosisCode_4         75.741351
ClmDiagnosisCode_5         85.640586
ClmDiagnosisCode_6         90.582864
ClmDiagnosisCode_7         93.633640
ClmDiagnosisCode_8         95.574587
ClmDiagnosisCode_9         97.134066
ClmDiagnosisCode_10        99.790820
ClmProcedureCode_1         99.968710
ClmProcedureCode_2         99.993047
ClmProcedureCode_3         99.999227
ClmProcedureCode_4         99.999614
ClmProcedureCode_5        100.000000
ClmProcedureCode_6        100.000000
DeductibleAmtPaid           0.000000
ClmAdmitDiagno

#### 1. Num of outpatient claims for each provider

In [147]:
df13 = agg_calculation(Train_Outpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'ClaimID',
                    sort_col = 'op_claims_num',
                    agg_method_lst = ['count'],
                    col_name_lst = ['Provider', 'op_claims_num']
                       )
df13

,Provider,op_claims_num
0,PRV51459,8240
1,PRV53797,4739
2,PRV51574,4444
3,PRV53918,3588
4,PRV54895,3433
...,...,...
5007,PRV52754,1
5008,PRV56029,1
5009,PRV52757,1
5010,PRV56050,1


#### 2. Num of outpatient patients for each provider

In [148]:
df14 = agg_calculation(Train_Outpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'BeneID',
                    sort_col = 'op_patients_num',
                    agg_method_lst = ['nunique'],
                    col_name_lst = ['Provider', 'op_patients_num']
                       )
df_merged_op = df13.merge(df14, how = 'outer', on = 'Provider')
df_merged_op

,Provider,op_claims_num,op_patients_num
0,PRV51459,8240,2857
1,PRV53797,4739,2020
2,PRV51574,4444,1715
3,PRV53918,3588,1709
4,PRV54895,3433,1466
...,...,...,...
5007,PRV52754,1,1
5008,PRV56029,1,1
5009,PRV52757,1,1
5010,PRV56050,1,1


#### 3. Total outpatient reimbursement for each provider

In [149]:
df15 = agg_calculation(Train_Outpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'InscClaimAmtReimbursed',
                    sort_col = 'op_reimbursed',
                    agg_method_lst = ['sum'],
                    col_name_lst = ['Provider', 'op_reimbursed']
                       )
df_merged_op = df_merged_op.merge(df15, how = 'outer', on = 'Provider')
df_merged_op

,Provider,op_claims_num,op_patients_num,op_reimbursed
0,PRV51459,8240,2857,2321890
1,PRV53797,4739,2020,1303600
2,PRV51574,4444,1715,1281810
3,PRV53918,3588,1709,1014510
4,PRV54895,3433,1466,1004610
...,...,...,...,...
5007,PRV52754,1,1,10
5008,PRV56029,1,1,100
5009,PRV52757,1,1,50
5010,PRV56050,1,1,40


#### 4. Total outpatient deductible amount for each provider

In [150]:
df16 = agg_calculation(Train_Outpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'DeductibleAmtPaid',
                    sort_col = 'op_deductible',
                    agg_method_lst = ['sum'],
                    col_name_lst = ['Provider', 'op_deductible']
                       )
df_merged_op = df_merged_op.merge(df16, how = 'outer', on = 'Provider')
df_merged_op

,Provider,op_claims_num,op_patients_num,op_reimbursed,op_deductible
0,PRV51459,8240,2857,2321890,22790
1,PRV53797,4739,2020,1303600,14020
2,PRV51574,4444,1715,1281810,12290
3,PRV53918,3588,1709,1014510,9100
4,PRV54895,3433,1466,1004610,11760
...,...,...,...,...,...
5007,PRV52754,1,1,10,0
5008,PRV56029,1,1,100,0
5009,PRV52757,1,1,50,0
5010,PRV56050,1,1,40,0


#### 5. Total claim length for each provider

In [151]:
startdate = pd.to_datetime(Train_Outpatientdata.ClaimStartDt)
enddate = pd.to_datetime(Train_Outpatientdata.ClaimEndDt)
Train_Outpatientdata['Cperiod'] = (enddate - startdate).dt.days+1

df17 = agg_calculation(Train_Outpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'Cperiod',
                    sort_col = 'op_cperiod',
                    agg_method_lst = ['sum'],
                    col_name_lst = ['Provider', 'op_cperiod']
                       )
df_merged_op = df_merged_op.merge(df17, how = 'outer', on = 'Provider')
df_merged_op

,Provider,op_claims_num,op_patients_num,op_reimbursed,op_deductible,op_cperiod
0,PRV51459,8240,2857,2321890,22790,19782
1,PRV53797,4739,2020,1303600,14020,10595
2,PRV51574,4444,1715,1281810,12290,10767
3,PRV53918,3588,1709,1014510,9100,8344
4,PRV54895,3433,1466,1004610,11760,8139
...,...,...,...,...,...,...
5007,PRV52754,1,1,10,0,1
5008,PRV56029,1,1,100,0,1
5009,PRV52757,1,1,50,0,1
5010,PRV56050,1,1,40,0,1


#### 6. Calculate a relationship score among three physcian columns for each claim and average on each provider

In [152]:
Train_Outpatientdata['phy_same'] = Train_Outpatientdata.apply(physician_same, axis=1)

In [153]:
df18 = agg_calculation(Train_Outpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'phy_same',
                    sort_col = 'op_phy_same',
                    agg_method_lst = ['mean'],
                    col_name_lst = ['Provider', 'op_phy_same']
                       )
df_merged_op = df_merged_op.merge(df18, how = 'outer', on = 'Provider')
df_merged_op

,Provider,op_claims_num,op_patients_num,op_reimbursed,op_deductible,op_cperiod,op_phy_same
0,PRV51459,8240,2857,2321890,22790,19782,3.388835
1,PRV53797,4739,2020,1303600,14020,10595,3.241612
2,PRV51574,4444,1715,1281810,12290,10767,3.635464
3,PRV53918,3588,1709,1014510,9100,8344,3.503902
4,PRV54895,3433,1466,1004610,11760,8139,3.768715
...,...,...,...,...,...,...,...
5007,PRV52754,1,1,10,0,1,4.000000
5008,PRV56029,1,1,100,0,1,2.000000
5009,PRV52757,1,1,50,0,1,4.000000
5010,PRV56050,1,1,40,0,1,4.000000


#### 7. Number of different physicians who appear on a claim and average on each provider

In [154]:
#encoding types of physicians into numeric values
Train_Outpatientdata[['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']] = np.where(Train_Outpatientdata[['AttendingPhysician','OperatingPhysician','OtherPhysician']].isnull(), 0, 1)

# number of different physicians who attend a patient
Train_Outpatientdata['N_Types_Physicians'] = Train_Outpatientdata['AttendingPhysician'] +  Train_Outpatientdata['OperatingPhysician'] + Train_Outpatientdata['OtherPhysician']


In [155]:
df19 = agg_calculation(Train_Outpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'N_Types_Physicians',
                    sort_col = 'op_N_Types_Physicians',
                    agg_method_lst = ['mean'],
                    col_name_lst = ['Provider', 'op_N_Types_Physicians']
                       )
df_merged_op = df_merged_op.merge(df19, how = 'outer', on = 'Provider')
df_merged_op

,Provider,op_claims_num,op_patients_num,op_reimbursed,op_deductible,op_cperiod,op_phy_same,op_N_Types_Physicians
0,PRV51459,8240,2857,2321890,22790,19782,3.388835,1.560316
1,PRV53797,4739,2020,1303600,14020,10595,3.241612,1.568052
2,PRV51574,4444,1715,1281810,12290,10767,3.635464,1.568182
3,PRV53918,3588,1709,1014510,9100,8344,3.503902,1.544314
4,PRV54895,3433,1466,1004610,11760,8139,3.768715,1.555491
...,...,...,...,...,...,...,...,...
5007,PRV52754,1,1,10,0,1,4.000000,1.000000
5008,PRV56029,1,1,100,0,1,2.000000,2.000000
5009,PRV52757,1,1,50,0,1,4.000000,1.000000
5010,PRV56050,1,1,40,0,1,4.000000,1.000000


#### 8. top 5 ClmAdmitDiagnosisCode for each provider

In [156]:
df20 = Train_Outpatientdata.groupby('Provider').apply(lambda x: get_top_5_codes(x, 'ClmAdmitDiagnosisCode')).reset_index(name='Top_5_ClmAdmitDiagnosisCode_op')
df20

,Provider,Top_5_ClmAdmitDiagnosisCode_op
0,PRV51001,"[59970, 78902, 78900, 78659, 56722]"
1,PRV51003,"[2724, 7862, 38918, V7283, 7242]"
2,PRV51004,"[V7612, 2724, 78079, 78701, 5789]"
3,PRV51005,"[42731, V7612, 7242, 7295, 4011]"
4,PRV51007,"[25000, 71594, 78651, 7140, 71940]"
...,...,...
5007,PRV57759,"[78959, 20070, V5883]"
5008,PRV57760,"[7242, 51889, V7612, 3669, 25080]"
5009,PRV57761,"[7295, 87342, 7231, 4011, 7866]"
5010,PRV57762,[1748]


In [157]:
df_merged_op = df_merged_op.merge(df20, how = 'outer', on = 'Provider')
df_merged_op

,Provider,op_claims_num,op_patients_num,op_reimbursed,op_deductible,op_cperiod,op_phy_same,op_N_Types_Physicians,Top_5_ClmAdmitDiagnosisCode_op
0,PRV51459,8240,2857,2321890,22790,19782,3.388835,1.560316,"[V7612, 42731, 4019, 25000, V5883]"
1,PRV53797,4739,2020,1303600,14020,10595,3.241612,1.568052,"[V7612, 4019, 42731, V5883, 78900]"
2,PRV51574,4444,1715,1281810,12290,10767,3.635464,1.568182,"[V7612, 42731, 25000, 4019, 7295]"
3,PRV53918,3588,1709,1014510,9100,8344,3.503902,1.544314,"[V7612, 42731, 25000, 4019, 7862]"
4,PRV54895,3433,1466,1004610,11760,8139,3.768715,1.555491,"[V7612, 25000, 42731, 4019, 7295]"
...,...,...,...,...,...,...,...,...,...
5007,PRV52754,1,1,10,0,1,4.000000,1.000000,[]
5008,PRV56029,1,1,100,0,1,2.000000,2.000000,[]
5009,PRV52757,1,1,50,0,1,4.000000,1.000000,[]
5010,PRV56050,1,1,40,0,1,4.000000,1.000000,[]


#### 9. top 5 ClmDiagnosisCode for each provider

In [158]:
diagnosis_col = ['ClmDiagnosisCode_1','ClmDiagnosisCode_2','ClmDiagnosisCode_3','ClmDiagnosisCode_4','ClmDiagnosisCode_5','ClmDiagnosisCode_6','ClmDiagnosisCode_7','ClmDiagnosisCode_8','ClmDiagnosisCode_9','ClmDiagnosisCode_10']

df21 = Train_Outpatientdata.groupby('Provider').apply(lambda x: top_5_across_columns(x, diagnosis_col)).reset_index(name='Top_5_ClmDiagnosisCode_op')
df21

,Provider,Top_5_ClmDiagnosisCode_op
0,PRV51001,"[5168, V4581, 4019, 5953, 78650]"
1,PRV51003,"[V5869, 25000, 4019, 2720, 2809]"
2,PRV51004,"[4019, 2724, 25000, 2449, 2720]"
3,PRV51005,"[4019, 2724, 4011, V5869, 25000]"
4,PRV51007,"[4019, 2449, V5861, 25000, 2724]"
...,...,...
5007,PRV57759,"[V5869, 4019, 25000, 28521, 7084]"
5008,PRV57760,"[4019, 2722, 7904, 25000, 72401]"
5009,PRV57761,"[4019, 2449, V5861, 4011, 28521]"
5010,PRV57762,[2330]


In [159]:
df_merged_op = df_merged_op.merge(df21, how = 'outer', on = 'Provider')
df_merged_op

,Provider,op_claims_num,op_patients_num,op_reimbursed,op_deductible,op_cperiod,op_phy_same,op_N_Types_Physicians,Top_5_ClmAdmitDiagnosisCode_op,Top_5_ClmDiagnosisCode_op
0,PRV51459,8240,2857,2321890,22790,19782,3.388835,1.560316,"[V7612, 42731, 4019, 25000, V5883]","[4019, 2724, 25000, V5869, 4011]"
1,PRV53797,4739,2020,1303600,14020,10595,3.241612,1.568052,"[V7612, 4019, 42731, V5883, 78900]","[4019, 25000, 2724, V5869, 4011]"
2,PRV51574,4444,1715,1281810,12290,10767,3.635464,1.568182,"[V7612, 42731, 25000, 4019, 7295]","[4019, 25000, 2724, 4011, V5869]"
3,PRV53918,3588,1709,1014510,9100,8344,3.503902,1.544314,"[V7612, 42731, 25000, 4019, 7862]","[4019, 25000, 2724, 4011, V5869]"
4,PRV54895,3433,1466,1004610,11760,8139,3.768715,1.555491,"[V7612, 25000, 42731, 4019, 7295]","[4019, 25000, 2724, 4011, V5861]"
...,...,...,...,...,...,...,...,...,...,...
5007,PRV52754,1,1,10,0,1,4.000000,1.000000,[],"[2724, 462, 25000]"
5008,PRV56029,1,1,100,0,1,2.000000,2.000000,[],[4910]
5009,PRV52757,1,1,50,0,1,4.000000,1.000000,[],[]
5010,PRV56050,1,1,40,0,1,4.000000,1.000000,[],"[38600, 5932, 4019, V5869]"


### Beneficiary

In [160]:
print('inpatient columns:\n',  Train_Beneficiarydata.columns)
Train_Beneficiarydata.head(5)

inpatient columns:
 Index(['BeneID', 'DOB', 'DOD', 'Gender', 'Race', 'RenalDiseaseIndicator',
       'State', 'County', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
       'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke', 'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt'],
      dtype='object')


,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,...,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,...,1,1,1,2,1,1,36000,3204,60,70
1,BENE11002,1936-09-01,NaN,2,1,0,39,280,12,12,...,2,2,2,2,2,2,0,0,30,50
2,BENE11003,1936-08-01,NaN,1,1,0,52,590,12,12,...,2,2,1,2,2,2,0,0,90,40
3,BENE11004,1922-07-01,NaN,1,1,0,39,270,12,12,...,2,1,1,1,1,2,0,0,1810,760
4,BENE11005,1935-09-01,NaN,1,1,0,24,680,12,12,...,2,1,2,2,2,2,0,0,1790,1200


In [161]:
Train_Allpatientdata = pd.merge(Train_Outpatientdata, Train_Inpatientdata, how='outer')
Train_Allpatientdata = pd.merge(Train_Allpatientdata, Train_Beneficiarydata, on = 'BeneID', how = 'outer')

#### 1. Num of death cases

In [162]:
Train_Allpatientdata['Is_Dead'] = np.where(Train_Allpatientdata['DOD'].isnull(), 0, 1)

In [163]:
df22 = agg_calculation(Train_Allpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'Is_Dead',
                    sort_col = 'num_death_cases',
                    agg_method_lst = ['sum'],
                    col_name_lst = ['Provider', 'num_death_cases']
                       )
df22

,Provider,num_death_cases
0,PRV51459,52
1,PRV53918,35
2,PRV53797,35
3,PRV55215,31
4,PRV52064,31
...,...,...
5405,PRV53632,0
5406,PRV53635,0
5407,PRV53638,0
5408,PRV53640,0


#### 2. Average patients' age for each provider

maybe some providers are specialized on specific age disease

In [164]:
#Add column age
# Convert 'DOB' and 'ClaimStartDt' columns to datetime format
Train_Allpatientdata['DOB'] =  pd.to_datetime(Train_Allpatientdata['DOB'], format='%Y-%m-%d')  
Train_Allpatientdata['ClaimStartDt'] = pd.to_datetime(Train_Allpatientdata['ClaimStartDt'], format='%Y-%m-%d')  
Train_Allpatientdata['Age'] = (Train_Allpatientdata['ClaimStartDt'] - Train_Allpatientdata['DOB']).dt.days // 365

In [165]:
unique_bene_per_provider = Train_Allpatientdata.drop_duplicates(subset=['Provider', 'BeneID'])
df23 = agg_calculation(unique_bene_per_provider,
                    groupby_col = 'Provider',
                    agg_col = 'Age',
                    sort_col = 'avg_age',
                    agg_method_lst = ['mean'],
                    col_name_lst = ['Provider', 'avg_age']
                       )
df23

,Provider,avg_age
0,PRV55591,99.0
1,PRV54817,99.0
2,PRV55355,98.0
3,PRV51423,98.0
4,PRV56974,98.0
...,...,...
5405,PRV52816,40.0
5406,PRV53501,40.0
5407,PRV57473,37.0
5408,PRV51236,35.0


In [166]:
df_merged_pt = df22.merge(df23, how = 'outer', on = 'Provider')
df_merged_pt

,Provider,num_death_cases,avg_age
0,PRV51459,52,73.532027
1,PRV53918,35,72.317145
2,PRV53797,35,72.515842
3,PRV55215,31,72.106520
4,PRV52064,31,73.094401
...,...,...,...
5405,PRV53632,0,75.250000
5406,PRV53635,0,69.442308
5407,PRV53638,0,69.907407
5408,PRV53640,0,73.962963


#### 3. Num of different state (location of provider)

In [167]:
df24 = agg_calculation(Train_Allpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'State',
                    sort_col = 'num_State',
                    agg_method_lst = ['nunique'],
                    col_name_lst = ['Provider', 'num_State']
                       )
df24

,Provider,num_State
0,PRV51578,38
1,PRV54070,38
2,PRV52019,36
3,PRV55510,34
4,PRV52154,32
...,...,...
5405,PRV52179,1
5406,PRV52178,1
5407,PRV52176,1
5408,PRV52173,1


In [168]:
df_merged_pt = df_merged_pt.merge(df24, how = 'outer', on = 'Provider')
df_merged_pt

,Provider,num_death_cases,avg_age,num_State
0,PRV51459,52,73.532027,10
1,PRV53918,35,72.317145,5
2,PRV53797,35,72.515842,9
3,PRV55215,31,72.106520,25
4,PRV52064,31,73.094401,28
...,...,...,...,...
5405,PRV53632,0,75.250000,2
5406,PRV53635,0,69.442308,6
5407,PRV53638,0,69.907407,4
5408,PRV53640,0,73.962963,4


#### 4. Num of different county (location of provider)

In [169]:
df25 = agg_calculation(Train_Allpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'County',
                    sort_col = 'num_County',
                    agg_method_lst = ['nunique'],
                    col_name_lst = ['Provider', 'num_County']
                       )
df25

,Provider,num_County
0,PRV55215,112
1,PRV57173,103
2,PRV54070,96
3,PRV52019,88
4,PRV56444,87
...,...,...
5405,PRV52286,1
5406,PRV52294,1
5407,PRV52296,1
5408,PRV55895,1


In [170]:
df_merged_pt = df_merged_pt.merge(df25, how = 'outer', on = 'Provider')
df_merged_pt

,Provider,num_death_cases,avg_age,num_State,num_County
0,PRV51459,52,73.532027,10,24
1,PRV53918,35,72.317145,5,43
2,PRV53797,35,72.515842,9,25
3,PRV55215,31,72.106520,25,112
4,PRV52064,31,73.094401,28,82
...,...,...,...,...,...
5405,PRV53632,0,75.250000,2,5
5406,PRV53635,0,69.442308,6,12
5407,PRV53638,0,69.907407,4,9
5408,PRV53640,0,73.962963,4,4


#### 5. Average Charlson Comorbidity Index(CCI) of patients for each provider

In [175]:
# Replace values with a CCI score
Train_Allpatientdata = Train_Allpatientdata.replace({'ChronicCond_Alzheimer': 2, 'ChronicCond_Heartfailure': 2, 'ChronicCond_KidneyDisease': 2,
                   'ChronicCond_Cancer': 2, 'ChronicCond_ObstrPulmonary': 2, 'ChronicCond_Depression': 2,
                   'ChronicCond_Diabetes': 2, 'ChronicCond_IschemicHeart': 2, 'ChronicCond_Osteoporasis': 2,
                   'ChronicCond_rheumatoidarthritis': 2, 'ChronicCond_stroke': 2, 'RenalDiseaseIndicator': '0'}, 
                  0)

In [176]:
# CII score calculation
Train_Allpatientdata = Train_Allpatientdata.replace({'ChronicCond_KidneyDisease': 1}, 2)
Train_Allpatientdata = Train_Allpatientdata.replace({'ChronicCond_Cancer': 1}, 4)
Train_Allpatientdata = Train_Allpatientdata.replace({'ChronicCond_Diabetes': 1}, 1.5)
Train_Allpatientdata = Train_Allpatientdata.replace({'ChronicCond_Osteoporasis': 1}, 2)
Train_Allpatientdata = Train_Allpatientdata.replace({'RenalDiseaseIndicator': 'Y'}, 2)

Train_Allpatientdata['CII_age_score'] = np.where(Train_Allpatientdata['Age'] < 50, 0,
                                                 np.where((Train_Allpatientdata['Age'] >= 50) & (Train_Allpatientdata['Age'] <= 59), 1,
                                                          np.where((Train_Allpatientdata['Age'] >= 60) & (Train_Allpatientdata['Age'] <= 69), 2,
                                                                   np.where((Train_Allpatientdata['Age'] >= 70) & (Train_Allpatientdata['Age'] <= 79), 3,
                                                                            np.where(Train_Allpatientdata['Age'] >= 80, 4, np.nan)))))

In [177]:
cci_column = ['RenalDiseaseIndicator', 'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
            'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
            'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
            'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke', 'CII_age_score']

In [178]:
Train_Allpatientdata['cci_score'] = Train_Allpatientdata[cci_column].sum(axis = 1)

In [179]:
df26 = agg_calculation(Train_Allpatientdata,
                    groupby_col = 'Provider',
                    agg_col = 'cci_score',
                    sort_col = 'avg_cci_score',
                    agg_method_lst = ['mean'],
                    col_name_lst = ['Provider', 'avg_cci_score']
                       )
df26

,Provider,avg_cci_score
0,PRV54054,17.5
1,PRV55405,15.5
2,PRV52791,15.5
3,PRV56084,15.5
4,PRV55138,14.5
...,...,...
5405,PRV52638,2.0
5406,PRV54420,1.5
5407,PRV57467,1.0
5408,PRV51968,0.0


In [180]:
df_merged_pt = df_merged_pt.merge(df26, how = 'outer', on = 'Provider')
df_merged_pt

,Provider,num_death_cases,avg_age,num_State,num_County,avg_cci_score
0,PRV51459,52,73.532027,10,24,7.686772
1,PRV53918,35,72.317145,5,43,7.557832
2,PRV53797,35,72.515842,9,25,7.603186
3,PRV55215,31,72.106520,25,112,7.615827
4,PRV52064,31,73.094401,28,82,7.710267
...,...,...,...,...,...,...
5405,PRV53632,0,75.250000,2,5,9.400000
5406,PRV53635,0,69.442308,6,12,7.157407
5407,PRV53638,0,69.907407,4,9,8.072072
5408,PRV53640,0,73.962963,4,4,7.030303


### Merge

In [181]:
df_final = df_merged_pt.merge(df_merged_op, how = 'outer', on = 'Provider')

In [182]:
df_final = df_final.merge(df_merged, how = 'outer', on = 'Provider')
df_final

,Provider,num_death_cases,avg_age,num_State,num_County,avg_cci_score,op_claims_num,op_patients_num,op_reimbursed,op_deductible,...,ip_reimbursed,ip_deductible,ip_cperiod,ip_hperiod,ip_phy_same,ip_N_Types_Physicians,Top_5_ClmAdmitDiagnosisCode_ip,Top_5_DiagnosisGroupCode_ip,Top_5_ClmDiagnosisCode_ip,Top_5_ClmProcedureCode_ip
0,PRV51459,52,73.532027,10,24,7.686772,8240.0,2857.0,2321890.0,22790.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PRV53918,35,72.317145,5,43,7.557832,3588.0,1709.0,1014510.0,9100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PRV53797,35,72.515842,9,25,7.603186,4739.0,2020.0,1303600.0,14020.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PRV55215,31,72.106520,25,112,7.615827,3250.0,2517.0,927260.0,7670.0,...,1357300.0,151656.0,935.0,935.0,3.846154,1.783217,"[486, 78650, 5990, 78605, 78097]","[344, 309, 421, 078, 251]","[4019, 5990, 2724, 25000, 42731]","[4019.0, 66.0, 9904.0, 3995.0, 2724.0]"
4,PRV52064,31,73.094401,28,82,7.710267,2806.0,1499.0,769680.0,8440.0,...,350000.0,39516.0,223.0,223.0,4.000000,1.710526,"[5990, 71536, 5789, 78097, 4280]","[387, 480, 283, 514, 511]","[4019, 41401, 42731, 53081, 2724]","[3995.0, 4019.0, 3241.0, 9910.0, 8152.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV53632,0,75.250000,2,5,9.400000,13.0,10.0,7380.0,0.0,...,7000.0,2136.0,8.0,8.0,4.000000,1.500000,"[99672, 6826]","[220, 606]","[319, 4589, 6826, 412, 70719]",[3893.0]
5406,PRV53635,0,69.442308,6,12,7.157407,49.0,47.0,8660.0,200.0,...,82000.0,5340.0,23.0,23.0,1.600000,1.800000,"[78906, 7862, 1977, 41401, V5811]","[356, 179, 408, 261, 834]","[4019, 3051, 2724, 5601, 486]","[9607.0, 5024.0, 3615.0, 9229.0, 1629.0]"
5407,PRV53638,0,69.907407,4,9,8.072072,111.0,54.0,38040.0,140.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5408,PRV53640,0,73.962963,4,4,7.030303,33.0,27.0,5660.0,160.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
